In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [3]:
feat_cup = pd.read_csv('/kaggle/input/ab-it-cup/feat_cup.csv')
objects_cup = pd.read_csv('/kaggle/input/ab-it-cup/objects_cup.csv')
feat_dict_cup = pd.read_excel('/kaggle/input/ab-it-cup/feat_dict_cup.xlsx', index_col='name_eng')

In [4]:
feat_cup.shape

(291882, 293)

# Удаление пустых строк

In [108]:
def delete_empty_rows(df):
    rows_with_all_na = df.drop(['client_id', 'dt'], axis=1).isna().all(axis=1)
    return df[~rows_with_all_na]

In [110]:
delete_empty_rows(feat_cup).shape

(225263, 293)

# Заполнение пустых значений

In [185]:
num_features = feat_cup.columns[feat_cup.dtypes != object]
cat_features = feat_cup.columns[feat_cup.dtypes == object]

In [187]:
amount_columns_mask = feat_dict_cup.loc[feat_cup.columns]['name_rus'].str.startswith('Количество')
percent_columns_mask = feat_dict_cup.loc[feat_cup.columns]['name_rus'].str.startswith('Доля')
sum_columns_mask = feat_dict_cup.loc[feat_cup.columns]['name_rus'].str.startswith('Сумма')
average_columns_mask = feat_dict_cup.loc[feat_cup.columns]['name_rus'].str.startswith('Средн')

In [233]:
mask = np.ma.mask_or(amount_columns_mask, percent_columns_mask)
mask = np.ma.mask_or(mask, sum_columns_mask)
mask = np.ma.mask_or(mask, average_columns_mask)

In [234]:
columns_to_fill_with_zeros = feat_cup.columns[mask]
columns_to_fill_with_median = list(filter(lambda x: x in num_features, feat_cup.columns[~mask]))
columns_to_fill_with_mode = cat_features

In [235]:
def fill_values_with_zeros(df, cols):
    navals = {i: 0 for i in cols}
    return df.fillna(navals)

In [236]:
def fill_values_with_median(df, cols):
    navals = {i: df[i].median() for i in cols}
    return df.fillna(navals)

In [237]:
def fill_values_with_mode(df, cols):
    navals = {i: df[i].mode().values[0] for i in cols}
    return df.fillna(navals)

In [238]:
feat_dict_cup.loc[list(filter(lambda x: x in num_features, feat_cup.columns[~mask]))]['name_rus'].tolist()

['Признак наличия зарплатного счета у клиента на отчетную дату',
 'Возраст клиента на отчетную дату',
 'Срок жизни клиента в банке (в днях)',
 'Общий дебетовый оборот по накопительным счетам',
 'Общий среднемесячный доход',
 'Разница между средним кредитным и дебетовым оборотом по накопительным счетам',
 'Разница между средним кредитным и дебетовым оборотом по текущим счетам',
 'hdb_other_active_credits_count',
 'Максимальный кредитный лимит по любому активному продукту за все время|OR|Максимальный кредитный лимит по активным договорам',
 'Максимальный лимит по кредитному продукту типа Кредит за все время',
 'Минимальная ПСК по активным кредитным продуктам в других банках на дату рассчета',
 'Максимальная текущая задолженность по кредитному продукту типа Кредитная карта',
 'Максимальный лимит по активному кредитному продукту типа Ипотека',
 'Максимальный кредитный лимит по любому продукту за все время',
 'Максимальный лимит по кредитному продукту типа Ипотека за все время',
 'Максималь

In [239]:
feat_cup[list(filter(lambda x: x in num_features, feat_cup.columns[~mask]))].min()

accountsalary_flag                         0.0
age                                       14.0
life_time_days                             1.0
turn_save_db_sum_v2                        0.0
total_inc                               1000.0
diff_save_cr_db_turn              -415431426.0
diff_cur_cr_db_turn               -411879009.0
hdb_other_active_credits_count             0.0
hdb_bki_active_max_limit                   0.0
hdb_bki_total_pil_max_limit                0.0
hdb_other_active_min_psk               -3569.7
hdb_bki_active_cc_max_outstand             0.0
hdb_bki_active_ip_max_limit                0.0
hdb_bki_total_max_limit                    0.0
hdb_bki_total_ip_max_limit              2681.0
hdb_bki_active_ip_max_outstand             0.0
hdb_bki_active_pil_max_limit               0.0
hdb_bki_total_auto_max_limit               0.0
hdb_bki_active_cc_max_limit                0.0
hdb_bki_total_cnt                          0.0
hdb_bki_total_cc_max_limit                 0.0
hdb_bki_activ

# Пайплайн

In [240]:
preprocessed_dataset = delete_empty_rows(feat_cup)

In [241]:
preprocessed_dataset = fill_values_with_zeros(preprocessed_dataset, columns_to_fill_with_zeros)

In [242]:
preprocessed_dataset['hdb_relend_client_credits_count_grade'] = preprocessed_dataset['hdb_relend_client_credits_count_grade'].fillna('d')

In [243]:
preprocessed_dataset = fill_values_with_mode(preprocessed_dataset, columns_to_fill_with_mode)

In [244]:
preprocessed_dataset = fill_values_with_median(preprocessed_dataset, columns_to_fill_with_median)

In [247]:
preprocessed_dataset.isna().sum()[preprocessed_dataset.isna().sum() > 0] #Проверка

Series([], dtype: int64)